In [ ]:
# private dock midestuary
# Data points every 5 minuetes
# HDR is sea level

import pandas as pd
import scipy.io as sio
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re

def Array_To_DataFrame(matDF, varName) :
    npArray = np.array([])
    for i in range (matDF[varName].shape[0]) :
        npArray = np.concatenate((npArray, matDF[varName][i].flatten()))
    return npArray
def matFile_To_DF(matFile) :
    tempDF = sio.loadmat(matFile)
    matDataMidFiltered = {}
    for dictKey in tempDF.keys() :
        if isinstance(tempDF[dictKey], np.ndarray) :
            if tempDF[dictKey].shape[0] > 1 :
                matDataMidFiltered.update({dictKey : tempDF[dictKey]})

    matMidDF = pd.DataFrame({dictKey: np.array(dictValue).flatten() for dictKey, dictValue in matDataMidFiltered.items()})
    return matMidDF

# import files for 2012-2013 and extract sea level data
matDockDF = matFile_To_DF('2012-13\\P_processed_01.mat')
# import downriver current data


#mat lab date origin
origin = np.datetime64('0000-01-01', 'D') - np.timedelta64(1, 'D')

#convert DN to datetime
matDockDF['DN'] = (matDockDF['DN'] * np.timedelta64(24*3600000, 'ms') + origin + np.timedelta64(500, 'ms')).astype('datetime64[s]')


seaLevelfig = px.line(x = matDockDF['DN'], y = [matDockDF['Hdr'], matDockDF['Hur']], color_discrete_sequence= ["black", "blue"], title = "Sea Level")
seaLevelFigLabel = ['Downriver Sea Level', 'Upriver Sea Level']
for idx in range(len(seaLevelFigLabel)):
    seaLevelfig.data[idx].name = seaLevelFigLabel[idx]
    seaLevelfig.data[idx].hovertemplate = 'variable=' + seaLevelFigLabel[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    seaLevelfig.data[idx].legendgroup = seaLevelFigLabel[idx]


seaLevelfig.update_layout(title=dict(text= "Sea Level for 2012-2013", font=dict(size=25)))
seaLevelfig.update_xaxes(tickangle=30)
seaLevelfig.update_xaxes(rangeslider_visible=True)
seaLevelfig.update_xaxes(range = [pd.Timestamp('2012-12-11'),pd.Timestamp('2013-05-26')])
seaLevelfig.update_layout(xaxis_title="Date", yaxis_title="Sea Level (m)", legend_title="Locations")
seaLevelfig.show()